In [ ]:
from skimage.draw import disk
from skimage.util import random_noise
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
def experiment(
    radius=5,
    margin=5,
    n=500,
    noise=False,
):
    minpos = radius * 2
    maxpos = 82 - radius * 2
    encoder = tf.keras.applications.MobileNetV3Small(
        include_top=False,
        pooling="avg",
    )
    clf = KNeighborsClassifier(
        n_neighbors=5,
        metric="minkowski",
    )

    def generate_positions():
        x1 = np.random.randint(low=minpos, high=maxpos)
        x2 = np.random.randint(low=minpos, high=maxpos)
        y1 = np.random.randint(low=minpos, high=maxpos)
        y2 = np.random.randint(low=minpos, high=maxpos)
        return x1, x2, y1, y2

    def create_img(x1, x2, y1, y2):
        img = np.zeros((82, 82, 3), dtype=np.double)
        r1, c1 = disk((x1, y1), radius, shape=img.shape)
        r2, c2 = disk((x2, y2), radius, shape=img.shape)
        img[r1, c1, :] = (1, 0, 0)
        img[r2, c2, :] = (0, 1, 0)
        return np.uint8(img * 255)

    images = []
    labels = []
    neg_count = 0
    pos_count = 0
    complete = False
    while not complete:
        print(neg_count, pos_count, end="\r")
        x1, x2, y1, y2 = generate_positions()
        img = create_img(x1, x2, y1, y2)
        if noise:
            img = random_noise(img)
        dist = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
        if (dist > radius * 2 + margin) and neg_count < n:
            images.append(img)
            labels.append(0)
            neg_count += 1
        elif dist < radius and pos_count < n:
            images.append(img)
            labels.append(1)
            pos_count += 1
        if pos_count == n and neg_count == n:
            complete = True
    x = np.array(images)

    xR = x[:, :, :, 0:1].repeat(3, axis=-1)
    xG = x[:, :, :, 1:2].repeat(3, axis=-1)
    xB = x[:, :, :, 2:3].repeat(3, axis=-1)
    concat_features = np.concatenate(
        [
            encoder.predict(xR),
            encoder.predict(xG),
            encoder.predict(xB),
        ],
        axis=1,
    )
    composite_features = encoder.predict(x)
    concat_scores = cross_val_score(
        clf, concat_features, labels, cv=5, scoring="balanced_accuracy"
    )
    composite_scores = cross_val_score(
        clf, composite_features, labels, cv=5, scoring="balanced_accuracy"
    )
    return concat_scores, composite_scores

In [ ]:
radii = [3, 6, 9, 12]
composite_scores = []
concat_scores = []
for r in radii:
    _concat_scores, _composite_scores = experiment(radius=r)
    concat_scores.append(_concat_scores)
    composite_scores.append(_composite_scores)

In [ ]:
plt.errorbar(
    x=radii,
    y=np.array(composite_scores).mean(axis=1),
    yerr=np.array(composite_scores).std(axis=1),
    label="composite",
)
plt.errorbar(
    x=radii,
    y=np.array(concat_scores).mean(axis=1),
    yerr=np.array(concat_scores).std(axis=1),
    label="concat",
)
plt.legend()
plt.title("Colocation Detection w/ Gaussian Noise")
plt.ylabel("Accuracy")
plt.xlabel("Objects Size [px]")

In [ ]:
plt.errorbar(
    x=radii,
    y=np.array(composite_scores).mean(axis=1),
    yerr=np.array(composite_scores).std(axis=1),
    label="composite",
)
plt.errorbar(
    x=radii,
    y=np.array(concat_scores).mean(axis=1),
    yerr=np.array(concat_scores).std(axis=1),
    label="concat",
)
plt.legend()
plt.title("Colocation Detection")
plt.ylabel("Accuracy")
plt.xlabel("Objects Size [px]")